In [22]:
!pip -q install python-telegram-bot python-dotenv nest-asyncio

import os
import csv
import sqlite3
import logging
from datetime import datetime, timezone

import nest_asyncio
from dotenv import load_dotenv
from zoneinfo import ZoneInfo

from telegram import Update
from telegram.ext import (
    Application,
    CommandHandler,
    MessageHandler,
    ConversationHandler,
    ContextTypes,
    filters
)


In [23]:
import os

os.environ["BOT_TOKEN"] = "тут когда-то был токен😇"
os.environ["CURRENCY"] = "RUB"
os.environ["DATABASE_PATH"] = "bot.db"
os.environ["TIMEZONE"] = "Europe/Moscow"
os.environ["ADMIN_ID"] = "0"

print("BOT_TOKEN set:", bool(os.getenv("BOT_TOKEN")))
print("CURRENCY:", os.getenv("CURRENCY"))
print("TIMEZONE:", os.getenv("TIMEZONE"))


BOT_TOKEN set: True
CURRENCY: RUB
TIMEZONE: Europe/Moscow


In [24]:
load_dotenv()

class Config:
    BOT_TOKEN = os.getenv("BOT_TOKEN")
    ADMIN_ID = int(os.getenv("ADMIN_ID", "0"))
    DATABASE_PATH = os.getenv("DATABASE_PATH", "bot.db")
    CURRENCY = os.getenv("CURRENCY", "RUB")
    TIMEZONE = os.getenv("TIMEZONE", "Europe/Moscow")

    WEBHOOK_URL = os.getenv("WEBHOOK_URL", "")
    WEBHOOK_PORT = int(os.getenv("WEBHOOK_PORT", "8443"))

    @classmethod
    def validate(cls):
        if not cls.BOT_TOKEN:
            raise ValueError("BOT_TOKEN не установлен (.env или os.environ)")

Config.validate()
LOCAL_TZ = ZoneInfo(Config.TIMEZONE)

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    level=logging.INFO
)
logger = logging.getLogger("expenses-bot")
logger.info("Config loaded. DB=%s TZ=%s CUR=%s", Config.DATABASE_PATH, Config.TIMEZONE, Config.CURRENCY)


2026-01-21 14:42:59,020 - expenses-bot - INFO - Config loaded. DB=bot.db TZ=Europe/Moscow CUR=RUB


In [25]:
class DatabaseManager:
    def __init__(self, db_path: str = Config.DATABASE_PATH):
        self.db_path = db_path
        self.init_database()

    def get_connection(self):
        conn = sqlite3.connect(self.db_path)
        conn.row_factory = sqlite3.Row
        return conn

    def init_database(self):
        with self.get_connection() as conn:
            conn.execute("""
                CREATE TABLE IF NOT EXISTS users (
                    user_id INTEGER PRIMARY KEY,
                    username TEXT,
                    first_name TEXT,
                    last_name TEXT,
                    language_code TEXT,
                    is_bot BOOLEAN,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    last_activity TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            """)

            conn.execute("""
                CREATE TABLE IF NOT EXISTS messages (
                    message_id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER,
                    text TEXT,
                    message_date TIMESTAMP,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (user_id) REFERENCES users (user_id)
                )
            """)

            conn.execute("""
                CREATE TABLE IF NOT EXISTS expenses (
                    expense_id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER NOT NULL,
                    amount REAL NOT NULL,
                    category TEXT NOT NULL,
                    note TEXT,
                    created_at_utc INTEGER NOT NULL,
                    FOREIGN KEY (user_id) REFERENCES users (user_id)
                )
            """)
            conn.execute("CREATE INDEX IF NOT EXISTS idx_expenses_user_time ON expenses(user_id, created_at_utc)")

        logger.info("DB initialized: %s", self.db_path)

    def save_user(self, user):
        with self.get_connection() as conn:
            conn.execute("""
                INSERT OR REPLACE INTO users
                (user_id, username, first_name, last_name, language_code, is_bot, last_activity)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                user.id, user.username, user.first_name, user.last_name,
                user.language_code, user.is_bot, datetime.now()
            ))

    def save_message(self, user_id: int, text: str, message_date):
        with self.get_connection() as conn:
            conn.execute("""
                INSERT INTO messages (user_id, text, message_date)
                VALUES (?, ?, ?)
            """, (user_id, text, message_date))

    def add_expense(self, user_id: int, amount: float, category: str, note: str):
        ts_utc = int(datetime.now(timezone.utc).timestamp())
        with self.get_connection() as conn:
            conn.execute("""
                INSERT INTO expenses (user_id, amount, category, note, created_at_utc)
                VALUES (?, ?, ?, ?, ?)
            """, (user_id, amount, category, note, ts_utc))

    def get_expenses_between(self, user_id: int, ts_from_utc: int, ts_to_utc: int):
        with self.get_connection() as conn:
            return conn.execute("""
                SELECT expense_id, amount, category, note, created_at_utc
                FROM expenses
                WHERE user_id=? AND created_at_utc>=? AND created_at_utc<?
                ORDER BY created_at_utc DESC
            """, (user_id, ts_from_utc, ts_to_utc)).fetchall()

    def sum_total_between(self, user_id: int, ts_from_utc: int, ts_to_utc: int) -> float:
        with self.get_connection() as conn:
            row = conn.execute("""
                SELECT COALESCE(SUM(amount), 0) AS total
                FROM expenses
                WHERE user_id=? AND created_at_utc>=? AND created_at_utc<?
            """, (user_id, ts_from_utc, ts_to_utc)).fetchone()
        return float(row["total"]) if row else 0.0

    def sum_by_category_between(self, user_id: int, ts_from_utc: int, ts_to_utc: int):
        with self.get_connection() as conn:
            return conn.execute("""
                SELECT category, ROUND(SUM(amount), 2) AS total
                FROM expenses
                WHERE user_id=? AND created_at_utc>=? AND created_at_utc<?
                GROUP BY category
                ORDER BY total DESC
            """, (user_id, ts_from_utc, ts_to_utc)).fetchall()

db = DatabaseManager()


2026-01-21 14:43:37,951 - expenses-bot - INFO - DB initialized: bot.db


In [27]:
from datetime import datetime, timezone

def bounds_today():
    now_local = datetime.now(LOCAL_TZ)
    start_local = now_local.replace(hour=0, minute=0, second=0, microsecond=0)
    ts_from = int(start_local.astimezone(timezone.utc).timestamp())
    ts_to = int(datetime.now(timezone.utc).timestamp()) + 1
    title = f"Сегодня ({start_local.date().isoformat()})"
    return ts_from, ts_to, title

def bounds_month():
    now_local = datetime.now(LOCAL_TZ)
    start_local = now_local.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
    ts_from = int(start_local.astimezone(timezone.utc).timestamp())
    ts_to = int(datetime.now(timezone.utc).timestamp()) + 1
    title = f"Месяц ({start_local.strftime('%Y-%m')})"
    return ts_from, ts_to, title

def format_summary(title: str, total: float, by_cat_rows) -> str:
    if total <= 0:
        return f"{title}\n\nПока нет расходов."
    lines = [f"{title}", "", f"Итого: {total:.2f} {Config.CURRENCY}", "", "По категориям:"]
    for r in by_cat_rows:
        lines.append(f"• {r['category']}: {float(r['total']):.2f} {Config.CURRENCY}")
    return "\n".join(lines)


In [28]:
AMOUNT, CATEGORY, NOTE = range(3)

async def add_start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Введите сумму (например 350 или 350.50):")
    return AMOUNT

async def add_amount(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = (update.message.text or "").strip().replace(",", ".")
    try:
        amount = float(text)
        if amount <= 0:
            raise ValueError
    except ValueError:
        await update.message.reply_text("Нужно число > 0. Попробуй ещё раз:")
        return AMOUNT

    context.user_data["amount"] = amount
    await update.message.reply_text("Введите категорию (еда/такси/дом):")
    return CATEGORY

async def add_category(update: Update, context: ContextTypes.DEFAULT_TYPE):
    cat = (update.message.text or "").strip().lower()
    if not cat:
        await update.message.reply_text("Категория не может быть пустой. Введите категорию:")
        return CATEGORY
    context.user_data["category"] = cat
    await update.message.reply_text("Комментарий (или '-' чтобы пропустить):")
    return NOTE

async def add_note(update: Update, context: ContextTypes.DEFAULT_TYPE):
    note = (update.message.text or "").strip()
    if note == "-":
        note = ""

    user = update.effective_user
    db.save_user(user)
    db.add_expense(user.id, context.user_data["amount"], context.user_data["category"], note)

    msg = f"Добавлено: {context.user_data['amount']:.2f} {Config.CURRENCY} — {context.user_data['category']}"
    if note:
        msg += f" — {note}"

    context.user_data.clear()
    await update.message.reply_text(msg)
    return ConversationHandler.END

async def add_cancel(update: Update, context: ContextTypes.DEFAULT_TYPE):
    context.user_data.clear()
    await update.message.reply_text("Добавление отменено.")
    return ConversationHandler.END


In [32]:
HELP_TEXT = (
    "Учёт расходов (RUB)\n\n"
    "Команды:\n"
    "/start\n"
    "/help\n"
    "/add — добавить расход\n"
    "/today — за сегодня\n"
    "/month — за месяц\n"
    "/export — за месяц (файл)\n"
    "/cancel — отмена (add)\n"
)

async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.effective_user
    db.save_user(user)
    await update.message.reply_text(f"Привет, {user.first_name}! \n\n{HELP_TEXT}")

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(HELP_TEXT)

async def today_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.effective_user
    db.save_user(user)
    ts_from, ts_to, title = bounds_today()
    total = db.sum_total_between(user.id, ts_from, ts_to)
    by_cat = db.sum_by_category_between(user.id, ts_from, ts_to)
    await update.message.reply_text(format_summary(title, total, by_cat))

async def month_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.effective_user
    db.save_user(user)
    ts_from, ts_to, title = bounds_month()
    total = db.sum_total_between(user.id, ts_from, ts_to)
    by_cat = db.sum_by_category_between(user.id, ts_from, ts_to)
    await update.message.reply_text(format_summary(title, total, by_cat))

async def export_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.effective_user
    db.save_user(user)

    ts_from, ts_to, title = bounds_month()
    rows = db.get_expenses_between(user.id, ts_from, ts_to)

    if not rows:
        await update.message.reply_text("За этот месяц пока нет расходов.")
        return

    filename = f"expenses_{datetime.now(LOCAL_TZ).strftime('%Y_%m')}.csv"
    with open(filename, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["expense_id", "amount", "currency", "category", "note", "datetime_local"])
        for r in rows:
            dt_local = datetime.fromtimestamp(int(r["created_at_utc"]), tz=timezone.utc).astimezone(LOCAL_TZ)
            w.writerow([r["expense_id"], f"{float(r['amount']):.2f}", Config.CURRENCY, r["category"], r["note"], dt_local.isoformat()])

    await update.message.reply_document(document=open(filename, "rb"), filename=filename, caption=f"Экспорт: {title}")

async def echo_and_save_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.effective_user
    db.save_user(user)
    db.save_message(user.id, update.message.text, update.message.date)
    await update.message.reply_text("Сообщение сохранено. Для расходов используй /add")

async def error_handler(update: object, context: ContextTypes.DEFAULT_TYPE):
    logger.error("Ошибка: %s", context.error)


In [33]:
class TelegramBot:
    def __init__(self, token: str):
        self.application = Application.builder().token(token).build()
        self.setup_handlers()

    def setup_handlers(self):
        self.application.add_handler(CommandHandler("start", start_command))
        self.application.add_handler(CommandHandler("help", help_command))
        self.application.add_handler(CommandHandler("today", today_command))
        self.application.add_handler(CommandHandler("month", month_command))
        self.application.add_handler(CommandHandler("export", export_command))

        conv_handler = ConversationHandler(
            entry_points=[CommandHandler("add", add_start)],
            states={
                AMOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, add_amount)],
                CATEGORY: [MessageHandler(filters.TEXT & ~filters.COMMAND, add_category)],
                NOTE: [MessageHandler(filters.TEXT & ~filters.COMMAND, add_note)],
            },
            fallbacks=[CommandHandler("cancel", add_cancel)],
        )
        self.application.add_handler(conv_handler)

        self.application.add_handler(
            MessageHandler(filters.TEXT & ~filters.COMMAND, echo_and_save_message)
        )

        self.application.add_error_handler(error_handler)

    def run_polling(self):
        print("Бот запущен (polling)...")
        self.application.run_polling()


In [34]:
nest_asyncio.apply()

bot = TelegramBot(Config.BOT_TOKEN)
app = bot.application

async def start_bot():
    await app.initialize()
    await app.start()
    await app.updater.start_polling()
    print("Бот запущен. Остановить: await stop_bot()")

async def stop_bot():
    await app.updater.stop()
    await app.stop()
    await app.shutdown()
    print("Бот остановлен")

await start_bot()


2026-01-21 15:19:03,559 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8588939828:AAEov9bgrKEhrvaQ9qNbuTXRlukqOpkr6dI/getMe "HTTP/1.1 200 OK"
2026-01-21 15:19:03,562 - telegram.ext.Application - INFO - Application started
2026-01-21 15:19:03,616 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8588939828:AAEov9bgrKEhrvaQ9qNbuTXRlukqOpkr6dI/deleteWebhook "HTTP/1.1 200 OK"


Бот запущен. Остановить: await stop_bot()


2026-01-21 15:19:07,930 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot8588939828:AAEov9bgrKEhrvaQ9qNbuTXRlukqOpkr6dI/getUpdates "HTTP/1.1 409 Conflict"
2026-01-21 15:19:07,932 - telegram.ext.Updater - ERROR - Exception happened while polling for updates.
Traceback (most recent call last):
  File "/Users/dmitry/venv/lib/python3.13/site-packages/telegram/ext/_utils/networkloop.py", line 134, in network_retry_loop
    await do_action()
  File "/Users/dmitry/venv/lib/python3.13/site-packages/telegram/ext/_utils/networkloop.py", line 127, in do_action
    action_cb_task.result()
    ~~~~~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/futures.py", line 199, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/tasks.py", line 304, in __step_run_and_handle_result
